<a href="https://colab.research.google.com/github/JustinGoheen/notebooks/blob/main/textlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install PyTorch Lightning

If you are running this locally and haven't already installed PyTorch Lightning, you can do so with:

```sh
pip install pytorch-lightning
```

In [ ]:
%%capture
!pip install "pytorch-lightning>=2.0" -q

# Import PyTorch Lightning and TorchMetrics

In [ ]:
import pytorch_lightning as pl
import torchmetrics as tm

# Import the Transformer and Dataset

In [ ]:
from pytorch_lightning.demos.transformer import Transformer, WikiText2

# Import PyTorch

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

# Create the Custom LightningDataModule

In [ ]:
import multiprocessing
import os
from pathlib import Path

from torch.utils.data import DataLoader, random_split

from pytorch_lightning.utilities.types import EVAL_DATALOADERS, TRAIN_DATALOADERS


class WikiText2DataModule(pl.LightningDataModule):
    def __init__(
        self,
        num_workers: int = 2,
        data_dir: Path = Path("./tutorial_data"),
        block_size: int = 35,
        download: bool = True,
        train_size: float = 0.8,
    ) -> None:
        super().__init__()
        self.data_dir = data_dir
        self.block_size = block_size
        self.download = download
        self.num_workers = num_workers
        self.train_size = train_size

    def prepare_data(self) -> None:
        self.dataset = WikiText2(data_dir=self.data_dir, block_size=self.block_size, download=self.download)

    def setup(self, stage: str) -> None:
        if stage == "fit" or stage is None:
            train_size = int(len(self.dataset) * self.train_size)
            test_size = len(self.dataset) - train_size
            self.train_data, self.val_data = random_split(self.dataset, lengths=[train_size, test_size])
        if stage == "test" or stage is None:
            self.test_data = self.val_data

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(self.train_data, num_workers=self.num_workers)

    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.val_data, num_workers=self.num_workers)

    def test_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(self.test_data, num_workers=self.num_workers)


# Create the LanguageModel class using LightningModule

In [ ]:
class LanguageModel(pl.LightningModule):
    def __init__(self, vocab_size: int = 33278):
        super().__init__()
        self.model = Transformer(vocab_size=vocab_size)

    def forward(self, inputs, target):
        return self.model(inputs, target)

    def training_step(self, batch, batch_idx):
        inputs, target = batch
        output = self(inputs, target)
        loss = torch.nn.functional.nll_loss(output, target.view(-1))
        return loss

    def configure_optimizers(self):
        return torch.optim.SGD(self.model.parameters(), lr=0.1)

    def prepare_data(self):
        WikiText2(download=True)

# Create a Trainer

## Prepare the Dataset

In [ ]:
datamodule = WikiText2DataModule()

## Instantiate a Model Object

In [ ]:
model = LanguageModel()

## Instantiate a Trainer Object

In [ ]:
logsdir = "./lightning_logs"

In [ ]:
trainer = pl.Trainer(
    max_epochs=2,
    logger=pl.loggers.CSVLogger(logsdir),
    profiler=pl.profilers.SimpleProfiler(dirpath=logsdir)
)

## Train the Model

In [ ]:
from time import perf_counter

In [ ]:
t1 = perf_counter()
trainer.fit(model=model, datamodule=datamodule)
t2 = perf_counter()

In [ ]:
print(t2 - t1)